In [1]:
import psi4
import py3Dmol as p3d
from qcelemental import constants as qcc

## Example 1: **l-alanine** geometry optimization

In the following cells, we will learn how to download a geometry of a molecule (in this case l-alanine)
from pubchem, visualise its geometry, check its rotational constants, optimize its structure to a minimum
with quantum chemistry, and then save its absolute energy, rotational constants, and dipole moments.

For this, we'll use the quantum chemistry package [Psi4](https://doi.org/10.1063/5.0006002) to do the computational heavy lifting.

### Step 1.1: getting l-alanine from pubchem:

This is fairly easy. We tell `Psi4` to search for a geometry of `<molecule>` on pubchem 
using the `psi4.geometry("pubchem:<molecule>")` syntax. As we want to use the geometry 
later, we save it into a variable called `l_alanine`:

In [24]:
l_alanine = psi4.geometry("pubchem:l-alanine")

	Searching PubChem database for l-alanine (single best match returned)
	Found 1 result(s)


### Step 1.2: getting the properties of the pubchem version of l-alanine:
First, we want to visualise the molecule.
Here we define a function `drawPsi4Geom()` that takes one argument,
a molecule geometry in the Psi4 format, and adapts it for viewing using py3Dmol.
We can then re-use this function by calling `drawPsi4Geom(<saved molecule>)`,
or in the case of l-alanine, `drawPsi4Geom(l_alanine)`.

In [3]:
def drawPsi4Geom(mol):
    xyz = mol.save_string_xyz_file() 
    view = p3d.view(width=600, height=400)
    view.addModel(xyz, "xyz")
    view.setStyle({'stick':{}})
    view.zoomTo()
    return(view.show())

drawPsi4Geom(l_alanine)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We can also check the rotational constants of the original molecule from pubchem. The function
`<molecule>.rotational_constants().np` returns the rotational constants as a wavenumber ($\lambda$) in 
cm<sup>-1</sup>. For the rest of this module, we'd prefer to use rotational constants as frequencies ($f$) 
in MHz, therefore we need to convert between the two using:

\begin{align}
f = c \lambda
\end{align}

In [7]:
l_alanine.rotational_constants().np

array([0.17048314, 0.10211747, 0.07887041])

We can define another helper function to aid us. The module `qcc` contains all important physical constants,
and we can access the speed of light ($c$, in m/s) as `qcc.c`. We need to multiply that by 100 (m/s $\rightarrow$ cm/s), and then convert from Hz ( = s <sup>-1</sup>) to MHz.

In [25]:
def getRotConstMHz(mol):
    incm = mol.rotational_constants().np
    inHz = [i * qcc.c * 100 for i in incm]
    inMHz = [i/1e6 for i in inHz]
    return(inMHz)

pubchem_rot_consts = getRotConstMHz(l_alanine)
print(pubchem_rot_consts)

[5108.350381344177, 3206.1060370070363, 2215.7842244027775]


We unfortunately cannot check the dipole moment of the pubchem version, as for that we need the electron density. We will look at that using the optimized version of the molecule.

### Step 1.3: Optimizing the geometry with Psi4:
When we want to find the minimum energy conformation of any molecule, the process by
which we do that is called "geometry optimization". We will use the program Psi4 to do that, 
using a good but computationally cheap method: [PBEh-3c by Grimme et al.](https://doi.org/10.1063/1.4927476)
The pubchem geometry is close to this minimum, but each computational method is slightly different, 
so it makes sense to make sure we are at the true minimum, especially as we want the rotational constants.


In the next cell, we will optimize the l-alanine molecule using the PBEh-3c density functional. 
**This may take a few minutes, so do not panic.** The absolute energy of the optimized molecule
will be saved into `E`, and the wavefunction of the optimized molecule into the `wfn` object. The new geometry
can be accessed as `wfn.molecule()`.

Once finished, a message "Optimizer: Optimization complete!" will appear...

In [6]:
E, wfn = psi4.optimize("pbeh3c", mol=l_alanine, return_wfn=True)

Optimizer: Optimization complete!


Now that the above cell has finished, let us visualise the molecule and check whether the rotational 
constants have changed. Can you tell a visual difference?

In [26]:
drawPsi4Geom(wfn.molecule())
print("pubchem version:")
print(pubchem_rot_consts)
print("optimized version:")
print(getRotConstMHz(wfn.molecule()))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

pubchem version:
[5108.350381344177, 3206.1060370070363, 2215.7842244027775]
optimized version:
[5110.9559332869385, 3061.4047705713156, 2364.4754628364058]


As you see in the output above, there is a significant change, 
even though visually the change is almost imperceptible.

How about dipole moments? They are stored in the `wfn` object under 
the following three keys, in Debye:

In [45]:
dip_X = wfn.scalar_variable("SCF DIPOLE X")
dip_Y = wfn.scalar_variable("SCF DIPOLE Y")
dip_Z = wfn.scalar_variable("SCF DIPOLE Z")
print(dip_X, dip_Y, dip_Z)

-0.8395597594507623 -2.0425904321991455 -0.06753220122530776
